# Day 09. Exercise 04
# Pipelines and OOP

## 0. Imports

In [1]:
import pandas as pd
import numpy as np
from typing import Self
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from tqdm.notebook import tqdm
import joblib

## 1. Preprocessing pipeline

Create three custom transformers, the first two out of which will be used within a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

1. `FeatureExtractor()` class:
 - Takes a dataframe with `uid`, `labname`, `numTrials`, `timestamp` from the file [`checker_submits.csv`](https://drive.google.com/file/d/14voc4fNJZiLEFaZyd8nEG-lQt5JjatYw/view?usp=sharing).
 - Extracts `hour` from `timestamp`.
 - Extracts `weekday` from `timestamp` (numbers).
 - Drops the `timestamp` column.
 - Returns the new dataframe.


2. `MyOneHotEncoder()` class:
 - Takes the dataframe from the result of the previous transformation and the name of the target column.
 - Identifies all the categorical features and transforms them with `OneHotEncoder()`. If the target column is categorical too, then the transformation should not apply to it.
 - Drops the initial categorical features.
 - Returns the dataframe with the features and the series with the target column.


3. `TrainValidationTest()` class:
 - Takes `X` and `y`.
 - Returns `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` (`test_size=0.2`, `random_state=21`, `stratified`).


In [2]:
class FeatureExtractor:
	def __init__(self, df : pd.DataFrame):
		self.df = df

	def transform(self) -> Self:
		self.df['hour']=self.df['timestamp'].dt.hour
		self.df['weekday']=self.df['timestamp'].dt.weekday
		self.df.drop(columns=['timestamp'], inplace=True)
		return self
	
	def get_df(self) -> pd.DataFrame:
		return self.df

df = FeatureExtractor(pd.read_csv("../data/checker_submits.csv", parse_dates=['timestamp'])).transform().get_df()
df

,uid,labname,numTrials,hour,weekday
0,user_4,project1,1,5,4
1,user_4,project1,2,5,4
2,user_4,project1,3,5,4
3,user_4,project1,4,5,4
4,user_4,project1,5,5,4
...,...,...,...,...,...
1681,user_19,laba06s,9,20,3
1682,user_1,laba06s,6,20,3
1683,user_1,laba06s,7,20,3
1684,user_1,laba06s,8,20,3


In [3]:
class MyOneHotEncoder:
	def __init__(self, df : pd.DataFrame, target : str):
		self.df = df
		self.target = target

	def transform(self) -> Self:
		categorial_col = [col for col in self.df.columns if self.df[col].dtype == 'category' and col != self.target]
		encoder = OneHotEncoder(sparse_output=False)
		temp_data = encoder.fit_transform(self.df[categorial_col])
		temp_namecol = encoder.get_feature_names_out()
		temp_df = pd.DataFrame(temp_data, columns=temp_namecol)
		self.df=pd.concat([self.df, temp_df], axis=1).drop(columns=['uid', 'labname'])
		return self
	
	def get_df(self) -> pd.DataFrame:
		return self.df

df['uid'] = df['uid'].astype('category')
df['labname'] = df['labname'].astype('category')

df=MyOneHotEncoder(df, 'weekday').transform().get_df()
df

,numTrials,hour,weekday,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,...,labname_lab02,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1
0,1,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,9,20,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1682,6,20,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1683,7,20,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1684,8,20,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
class TrainValidationTest:
	def __init__(self,X,y):
		self.X=X
		self.y=y
	
	def split(self) -> tuple[np.array,np.array,np.array,np.array]:
		return train_test_split(self.X,self.y,test_size=0.2,random_state=21,stratify=self.y)

X=df.drop(columns=['weekday'])
y=df['weekday']

X_train, X_test, y_train, y_test = TrainValidationTest(X,y).split()

## 2. Model selection pipeline

`ModelSelection()` class

 - Takes a list of `GridSearchCV` instances and a dict where the keys are the indexes from that list and the values are the names of the models, the example is below in the reverse order (from high-level to low-level perspective):

```
ModelSelection(grids, grid_dict)

grids = [gs_svm, gs_tree, gs_rf]

gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=jobs), where jobs you can specify by yourself

svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 'C':[0.01, 0.1, 1, 1.5, 5, 10], 'gamma': ['scale', 'auto'], 'class_weight':('balanced', None), 'random_state':[21], 'probability':[True]}]
```

 - Method `choose()` takes `X_train`, `y_train`, `X_valid`, `y_valid` and returns the name of the best classifier among all the models on the validation set
 - Method `best_results()` returns a dataframe with the columns `model`, `params`, `valid_score` where the rows are the best models within each class of models.

```
model	params	valid_score
0	SVM	{'C': 10, 'class_weight': None, 'gamma': 'auto...	0.877778
1	Decision Tree	{'class_weight': 'balanced', 'criterion': 'gin...	0.866667
2	Random Forest	{'class_weight': None, 'criterion': 'entropy',...	0.907407
```

 - When you iterate through the parameters of a model class, print the name of that class and show the progress using `tqdm.notebook`, in the end of the cycle print the best model of that class.

```
Estimator: SVM
100%
125/125 [01:32<00:00, 1.36it/s]
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878 

Estimator: Decision Tree
100%
57/57 [01:07<00:00, 1.22it/s]
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.867 

Estimator: Random Forest
100%
284/284 [06:47<00:00, 1.13s/it]
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907 

Classifier with best validation set accuracy: Random Forest
```

In [5]:
model_svm = SVC(probability=True, random_state=21)
svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 
               'C':[0.01, 0.1, 1, 1.5, 5, 10], 
               'gamma': ['scale', 'auto'], 
               'class_weight':('balanced', None), 
               'random_state':[21], 
               'probability':[True]}]
gs_svm =  GridSearchCV(estimator=model_svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=-1)


In [6]:
model_tree = DecisionTreeClassifier(random_state=21)
tree_params = [ {'max_depth': [1,5,10,20,30,40,45,49],
    'class_weight': ['balanced', None],
    'criterion':['entropy','gini']}]
gs_tree = GridSearchCV(estimator=model_tree, param_grid=tree_params, scoring='accuracy', cv=2, n_jobs=-1)

In [7]:
model_forest = RandomForestClassifier(random_state=21)
forest_params = [ {
    'max_depth': [1,5,10,20,30,40,45,49],
    'n_estimators' : [5,10,50,100],
    'class_weight': ['balanced', None],
    'criterion':['entropy','gini']
}]
gs_rf = GridSearchCV(estimator=model_forest, param_grid=forest_params, scoring='accuracy', cv=2, n_jobs=-1)

In [8]:
grids = [gs_svm, gs_tree, gs_rf]
grid_dict = {0 : 'SVC', 1: 'DecisionTreeClassifier', 2 : 'RandomForestClassifier'}

In [9]:
class ModelSelection():
    def __init__(self, grids, grid_dict):
        self.grids = grids
        self.grid_dict = grid_dict
    def choose(self, X_train, y_train, X_valid, y_valid):
        best_model_name = None
        best_score = 0
        for i, grid in enumerate(self.grids):
            model_name = self.grid_dict[i]
            print(f"Estimator {model_name}")
            total_fits = np.prod([len(v) for v in grid.param_grid[0].values()]) * 2
            with tqdm(total=total_fits, desc=f"{model_name}", ncols=100, unit="it/s") as pbar:
                grid.fit(X_train, y_train)
                
                valid_score = grid.score(X_valid, y_valid)
                if valid_score > best_score:
                    best_score = valid_score
                    best_model_name = model_name

                pbar.update(total_fits)    
                print(f"Best params: {grid.best_params_}")
                print(f"Best training accuracy: {grid.best_score_:.3f}")
                print(f"Validation set accuracy score for best params: {valid_score:.3f}")

        print(f"\nClassifier with best validation set accuracy: {best_model_name}")
        return best_model_name
    
    def best_results(self, X_train, y_train, X_valid, y_valid):
        models = []
        params =  []
        scores = []
        for i, grid in enumerate(self.grids):
            model_name = self.grid_dict[i]
            models.append(model_name)
            grid.fit(X_train, y_train)
            params.append(grid.best_params_)
            scores.append(grid.score(X_valid, y_valid))
        return pd.DataFrame({'model': models, 'param' : params, 'valid_score': scores})

In [10]:
grid_class = ModelSelection(grids, grid_dict)

In [11]:
grid_class.choose(X_train, y_train, X_test, y_test)

Estimator SVC


SVC:   0%|                                                                | 0/144 [00:00<?, ?it/s/s]

Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.809
Validation set accuracy score for best params: 0.888
Estimator DecisionTreeClassifier


DecisionTreeClassifier:   0%|                                              | 0/64 [00:00<?, ?it/s/s]

Best params: {'class_weight': None, 'criterion': 'gini', 'max_depth': 30}
Best training accuracy: 0.829
Validation set accuracy score for best params: 0.867
Estimator RandomForestClassifier


RandomForestClassifier:   0%|                                             | 0/256 [00:00<?, ?it/s/s]

Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 30, 'n_estimators': 100}
Best training accuracy: 0.874
Validation set accuracy score for best params: 0.932

Classifier with best validation set accuracy: RandomForestClassifier


'RandomForestClassifier'

In [12]:
grid_class.best_results(X_train, y_train, X_test, y_test)

,model,param,valid_score
0,SVC,"{'C': 10, 'class_weight': None, 'gamma': 'auto...",0.887574
1,DecisionTreeClassifier,"{'class_weight': None, 'criterion': 'gini', 'm...",0.866864
2,RandomForestClassifier,"{'class_weight': None, 'criterion': 'entropy',...",0.931953


## 3. Finalization

`Finalize()` class
 - Takes an estimator.
 - Method `final_score()` takes `X_train`, `y_train`, `X_test`, `y_test` and returns the accuracy of the model as in the example below:
```
final.final_score(X_train, y_train, X_test, y_test)
Accuracy of the final model is 0.908284023668639
```
 - Method `save_model()` takes a path, saves the model to this path and prints that the model was successfully saved.

In [13]:
class Finalize():
    def __init__(self, estimator):
        self.estimator = estimator
    def final_score(self, X_train, y_train, X_test, y_test):
        self.estimator.fit(X_train, y_train)
        y_pred = self.estimator.predict(X_test)
        print(f"Accuracy of the final model is {accuracy_score(y_test, y_pred)}")
        return accuracy_score(y_test, y_pred)
    def save_model(self, path):
        if joblib.dump(self.estimator, f'{path}'):
            print(f"Model was successfuly saved on path: {path}")
        else:
            raise Exception("Error while saving model")
model_final = Finalize(model_forest)
model_final.final_score(X_train, y_train, X_test, y_test)
model_final.save_model('model_ex04.joblib')

Accuracy of the final model is 0.9378698224852071
Model was successfuly saved on path: model_ex04.joblib


## 4. Main program

1. Load the data from the file (****name of file****).
2. Create the preprocessing pipeline that consists of two custom transformers: `FeatureExtractor()` and `MyOneHotEncoder()`:
```
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
```
3. Use that pipeline and its method `fit_transform()` on the initial dataset.
```
data = preprocessing.fit_transform(df)
```
4. Get `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` using `TrainValidationTest()` and the result of the pipeline.
5. Create an instance of `ModelSelection()`, use the method `choose()` applying it to the models that you want and parameters that you want, get the dataframe of the best results.
6. create an instance of `Finalize()` with your best model, use method `final_score()` and save the model in the format: `name_of_the_model_{accuracy on test dataset}.sav`.

That is it, congrats!

In [17]:
class FeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_new = X.copy() 
        X_new['timestamp'] = pd.to_datetime(X_new['timestamp'])
        X_new['hour'] = X_new['timestamp'].dt.hour  
        X_new['weekday'] = X_new['timestamp'].dt.weekday  
        X_new = X_new.drop(columns='timestamp')
        return X_new
class MyOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, target):
        self.target = target

    def fit(self, X, y=None):
        self.encoder = OneHotEncoder(sparse_output=False)
        self.categorical_cols = [col for col in X.columns if X[col].dtype == 'category' and col != self.target]
        self.encoder.fit(X[self.categorical_cols])
        return self

    def transform(self, X):
        encoded_features = self.encoder.transform(X[self.categorical_cols])
        encoded_columns = self.encoder.get_feature_names_out(self.categorical_cols)
        encoded_df = pd.DataFrame(encoded_features, columns=encoded_columns)
        X_new = pd.concat([X, encoded_df], axis=1).drop(columns=self.categorical_cols)
        return X_new

In [18]:
data = pd.read_csv('../data/checker_submits.csv')
data['uid'] = data["uid"].astype('category')
data['labname'] = data["labname"].astype('category')
data

,uid,labname,numTrials,timestamp
0,user_4,project1,1,2020-04-17 05:19:02.744528
1,user_4,project1,2,2020-04-17 05:22:45.549397
2,user_4,project1,3,2020-04-17 05:34:24.422370
3,user_4,project1,4,2020-04-17 05:43:27.773992
4,user_4,project1,5,2020-04-17 05:46:32.275104
...,...,...,...,...
1681,user_19,laba06s,9,2020-05-21 20:01:48.959966
1682,user_1,laba06s,6,2020-05-21 20:18:54.487900
1683,user_1,laba06s,7,2020-05-21 20:19:06.872761
1684,user_1,laba06s,8,2020-05-21 20:22:41.877806


In [19]:
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
df = preprocessing.fit_transform(data)

In [22]:
train = TrainValidationTest(X,y)
X_train, X_test, y_train, y_test = train.split()

In [23]:

grids = [gs_svm, gs_tree, gs_rf]
grid_dict = {0 : 'SVC', 1: 'DecisionTreeClassifier', 2 : 'RandomForestClassifier'}
grid_class = ModelSelection(grids, grid_dict)

In [24]:
grid_class.choose(X_train, y_train, X_test, y_test)

Estimator SVC


SVC:   0%|                                                                | 0/144 [00:00<?, ?it/s/s]

Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.809
Validation set accuracy score for best params: 0.888
Estimator DecisionTreeClassifier


DecisionTreeClassifier:   0%|                                              | 0/64 [00:00<?, ?it/s/s]

Best params: {'class_weight': None, 'criterion': 'gini', 'max_depth': 30}
Best training accuracy: 0.829
Validation set accuracy score for best params: 0.867
Estimator RandomForestClassifier


RandomForestClassifier:   0%|                                             | 0/256 [00:00<?, ?it/s/s]

Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 30, 'n_estimators': 100}
Best training accuracy: 0.874
Validation set accuracy score for best params: 0.932

Classifier with best validation set accuracy: RandomForestClassifier


'RandomForestClassifier'

In [25]:
grid_class.best_results(X_train, y_train, X_test, y_test)

,model,param,valid_score
0,SVC,"{'C': 10, 'class_weight': None, 'gamma': 'auto...",0.887574
1,DecisionTreeClassifier,"{'class_weight': None, 'criterion': 'gini', 'm...",0.866864
2,RandomForestClassifier,"{'class_weight': None, 'criterion': 'entropy',...",0.931953


In [26]:
final_model = Finalize(model_forest)
accuracy = final_model.final_score(X_train, y_train, X_test, y_test)

Accuracy of the final model is 0.9378698224852071


In [27]:
final_model.save_model(f'final_model_{accuracy:.3f}.sav')

Model was successfuly saved on path: final_model_0.938.sav
